In [ ]:
## Auto Turtel

import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time, date
import datetime as dt
import matplotlib.pyplot as plt
from my_functions3 import *
import warnings
warnings.filterwarnings('ignore')

import re

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
X,y = make_classification(n_samples= 1000,n_features= 10 , n_informative= 2, n_redundant=8,
                         weights=[0.9,0.1],  flip_y= 0,random_state= 42)

In [3]:
np.unique(y,return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

In [5]:
# define hyperparameter 

params = {
    "solver":"lbfgs",
    "max_iter":1000,
    "multi_class":"auto",
    "random_state":8888,
}

lr = LogisticRegression(**params)

In [6]:
lr.fit(X_train,y_train)

LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [7]:
y_pred = lr.predict(X_test)


In [8]:
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [9]:
report_dict = classification_report(y_test , y_pred ,output_dict= True)

In [10]:
####### mlflow

In [14]:
import mlflow

In [16]:
mlflow.set_experiment('First Experiment')
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0':report_dict['1']['recall'],
        'f1_score_macro':report_dict['macro avg']['f1-score']
    })
    
    mlflow.sklearn.log_model(lr,"logistic Rregressioin")


2024/10/30 13:15:26 INFO mlflow.tracking.fluent: Experiment with name 'First Experiment' does not exist. Creating a new experiment.
2024/10/30 13:15:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/30 13:15:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run painted-fish-415 at: http://127.0.0.1:5000/#/experiments/526862411928659788/runs/e01bcc8ff2d740f5b6cdc26018de1d30.
2024/10/30 13:15:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/526862411928659788.


## Experiment 1

In [20]:
log_reg = LogisticRegression(C =1 ,solver= 'liblinear')
log_reg.fit(X_train , y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test,y_pred_log_reg))


              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



## Experimnet 2

In [21]:
rf_clf = RandomForestClassifier(n_estimators= 30 ,max_depth= 3)
rf_clf.fit(X_train,y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test ,y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



## Experiment 3

In [22]:
xgb_clf =  XGBClassifier(use_label_encoder = False , eval_metrics = 'logos')
xgb_clf.fit(X_train,y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test,y_pred_xgb))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [5]:
models = [
    (
        "Logistic Regression",
        LogisticRegression(C=1,solver= 'liblinear'),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "Random Forest",
        RandomForestClassifier(n_estimators= 30,max_depth=3),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "XGBclassifier",
        XGBClassifier(use_label_encoder = False,eval_metric = 'logloss'),
        (X_train,y_train),
        (X_test,y_test)
        
    )
]

In [6]:
reports = []
for model_name ,model,train_set,test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test  = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train ,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test,y_pred,output_dict= True)
    reports.append(report)

In [7]:
import mlflow


In [12]:
mlflow.set_experiment("Anamoli Detection")
mlflow.set_tracking_uri(uri= "http://127.0.0.1:5000")

for i , element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=  model_name):
        mlflow.log_param('model_name',model_name)
        mlflow.log_metric('accuracy',report['accuracy'])
        mlflow.log_metric('recall_class_1' , report['1']['recall'])
        mlflow.log_metric("recall_class_0" , report['0']['recall'])
        mlflow.log_metric('f1_score_macro',report['macro avg']['f1-score'])
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model,"model")
        else:
            mlflow.sklearn.log_model(model,"model")
        

2024/10/30 20:23:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/30 20:23:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/462219254129592366/runs/1872a6461a624f42ada2b70ca39dbbab.
2024/10/30 20:23:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/462219254129592366.
2024/10/30 20:23:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/30 20:23:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/462219254129592366/runs/c0a624e9029b4925b5d17c895e3e18ca.
2024/10/30 20:23:34 INFO mlflow.tracking._tracking_service.client: 🧪 View exp

## Register model

In [29]:
model_name = "XGB-Smote"
run_id = input("Enter the run ID")
model_uri = f"r:/{run_id}/"

result =mlflow.register_model(
    model_uri,model_name
    
)

Enter the run ID8c2830ff53804614b9be03ac3f657adc


Successfully registered model 'XGB-Smote'.


RestException: INVALID_PARAMETER_VALUE: Invalid model version source: 'r:/8c2830ff53804614b9be03ac3f657adc/'. To use a local path as a model version source, the run_id request parameter has to be specified and the local path has to be contained within the artifact directory of the run specified by the run_id.

In [36]:
model_name = "XGB-Smote"
run_id = input("Enter the run ID")
model_uri = f"runs:/{run_id}/model"

result =mlflow.register_model(
    model_uri,model_name
    
)

Enter the run ID8c2830ff53804614b9be03ac3f657adc


Successfully registered model 'XGB-Smote'.
2024/11/01 11:32:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


## Load Model

In [38]:
model_name = "XGB-smote"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}" # if u want to load on the name of alias then @challenger add instedof /{model_verison}
loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [40]:
dev_model_uri = f"models:/{model_name}/{model_version}"
prod_model = "anomaly-detection-prod"


client = mlflow.MlflowClient()
client.copy_model_version(  src_model_uri= dev_model_uri , dst_name= prod_model )

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1730441261986, current_stage='None', description='', last_updated_timestamp=1730441261986, name='anomaly-detection-prod', run_id='8c2830ff53804614b9be03ac3f657adc', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>